## Title

## Introduction

## Prelim Analysis

### Libraries to load

In [11]:
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)

── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.5     ✔ rsample      1.2.0
✔ dials        1.2.0     ✔ tune         1.1.2
✔ infer        1.0.5     ✔ workflows    1.1.3
✔ modeldata    1.2.0     ✔ workflowsets 1.0.1
✔ parsnip      1.1.1     ✔ yardstick    1.2.0
✔ recipes      1.0.8     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks readr::spec()
✖ recipes::step()   masks stats::step()
• Learn how to get started at https://www.tidymodels.org/start/



In [14]:
# downloading and reading data, naming columns
url <- "https://raw.githubusercontent.com/ssun05/dsci_100_002_group5_project/main/processed.switzerland.data"
# download.file(url, "heart_data.csv")
heart_data <- read_csv("heart_data.csv", col_names = c("age", "sex", "chest_pain", "trestbps", "chol", "fasting_bld_sgar",
                                                      "restecg", "max_heart_rate", "exerc_angina", "oldpeak", "slope", "ca", "thal", "diagnosis"))

heart_data  

Rows: 123 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (9): trestbps, fasting_bld_sgar, restecg, max_heart_rate, exerc_angina, ...
dbl (5): age, sex, chest_pain, chol, diagnosis

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


age,sex,chest_pain,trestbps,chol,fasting_bld_sgar,restecg,max_heart_rate,exerc_angina,oldpeak,slope,ca,thal,diagnosis
<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
32,1,1,95,0,?,0,127,0,.7,1,?,?,1
34,1,4,115,0,?,?,154,0,.2,1,?,?,1
35,1,4,?,0,?,0,130,1,?,?,?,7,3
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
72,1,3,160,0,?,2,114,0,1.6,2,2,?,0
73,0,3,160,0,0,1,121,0,0,1,?,3,1
74,1,2,145,0,?,1,123,0,1.3,1,?,?,1


In [18]:
# choosing sex (could result in physiological differences in predictors)
# note that 1 is male and 0 is female
sex_count <- heart_data |> 
            group_by(sex) |>
            summarize(count = n())

sex_count
# 113 males and 10 females, males better sample size

sex,count
<dbl>,<int>
0,10
1,113


In [25]:
# filter for males and reclass diagnosis as factor, make labels understandable
heart_data_males <- heart_data |>
                    filter(sex == 1) |> 
                    mutate(diagnosis = as_factor(diagnosis)) |>
                     mutate(diagnosis = fct_recode(diagnosis, "none" = "0"))

heart_data_males

age,sex,chest_pain,trestbps,chol,fasting_bld_sgar,restecg,max_heart_rate,exerc_angina,oldpeak,slope,ca,thal,diagnosis
<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<fct>
32,1,1,95,0,?,0,127,0,.7,1,?,?,1
34,1,4,115,0,?,?,154,0,.2,1,?,?,1
35,1,4,?,0,?,0,130,1,?,?,?,7,3
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
70,1,4,140,0,1,0,157,1,2,2,?,7,3
72,1,3,160,0,?,2,114,0,1.6,2,2,?,none
74,1,2,145,0,?,1,123,0,1.3,1,?,?,1


In [26]:
# table of diagnosis counts
label_counts <- heart_data_males |>
                group_by(diagnosis) |>
                summarize(count = n())

label_counts

diagnosis,count
<fct>,<int>
none,8
1,42
2,29
3,29
4,5


In [3]:
# summary - eg. predictor distributions

## Methods

## Expected Outcomes and Significance